In [45]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [46]:
DATA_PATH = "../data"
DATA_FILENAME = "out_file_by_inn_223_44_last.csv"
data_file = os.path.join(DATA_PATH, DATA_FILENAME)
dict_0_30000_path = os.path.join(DATA_PATH, "0_30000.pickle")

In [39]:
iter_csv = pd.read_csv(data_file, encoding="utf-16", sep='\t', iterator=True, chunksize=1000000)
df = pd.concat([chunk[(chunk['Закупки по'] == '44-ФЗ') 
                & (chunk['Этап закупки']=='Определение поставщика завершено')
                & (chunk["Дата обновления"].str.split('.', expand=True).iloc[:, 2].astype("int").isin([2019, 2020, 2021]))] 
                for chunk in tqdm(iter_csv)])

11it [04:29, 24.48s/it]


In [47]:
with open(dict_0_30000_path, 'rb') as handle:
    d = pickle.load(handle)


In [130]:
df['purch'] = df['Реестровый номер закупки'].apply(lambda x: x[1:])
df["winer"] = df["purch"].apply(lambda p: d[p] if p in d else 0)
df = df[df.winer != 0]
inns = df.groupby("winer").winer.count()[df.groupby("winer").winer.count() > 15].index
df_val = df[df["winer"].isin(inns)]


In [131]:
df_val["Дата обновления"] = pd.to_datetime(df_val["Дата обновления"], format="%d.%m.%Y")
df_val["Год обновления"] = df_val["Дата обновления"].apply(lambda date: date.year)

In [165]:
df_val.to_csv(os.path.join(DATA_PATH, "valid.csv"), encoding="utf-8", sep=',')

In [132]:
stat = df_val.groupby(["winer", "Год обновления"])["Закупки по"].count()
stat

winer       Год обновления
0276065541  2019              13
            2020              11
            2021               9
0326040626  2020               8
            2021              17
                              ..
9721089700  2021              18
9731032950  2020              21
            2021              25
9731038990  2020              16
            2021              30
Name: Закупки по, Length: 169, dtype: int64

In [142]:
valid_inns = []
for inn in inns:
    inn_stat = stat[inn]
    if inn_stat.shape[0] == 1:
        valid_inns.append(inn)
        continue
    f = True
    for year in inn_stat.index[1:]:
        try:
            if (inn_stat[year] / inn_stat[year - 1]) < 0.75:
                f = False
                break
        except KeyError:
            f = False
    if f: valid_inns.append(inn)


In [143]:
len(valid_inns)

42

In [144]:
valid_inns

['0276065541',
 '0326040626',
 '2312178145',
 '2540203506',
 '2636209450',
 '3123470210',
 '3444197347',
 '3459070488',
 '4719016624',
 '5007103318',
 '5036171778',
 '5903027281',
 '5903134283',
 '6166084084',
 '6230114532',
 '7701213835',
 '7705970498',
 '7708823950',
 '7711027436',
 '7713056834',
 '7714511476',
 '7715656026',
 '7716874274',
 '7718538045',
 '7718861376',
 '7719481754',
 '7724419198',
 '7724490000',
 '7724922443',
 '7726311464',
 '7726416643',
 '7731345620',
 '7733271117',
 '7734420273',
 '7743529527',
 '8604037576',
 '9701158070',
 '9715366867',
 '9718148546',
 '9721089700',
 '9731032950',
 '9731038990']